In [1]:
import numpy as np
import pandas as pd
import dv_processing as dv

from datetime import timedelta

In [2]:
def extract_event_data(events):
    return np.array([[event.timestamp(),
                      event.x(),
                      event.y(),
                      event.polarity()] for event in events])


def get_event_data(file_path):
    capture = dv.io.MonoCameraRecording(file_path)

    slicer = dv.EventStreamSlicer()
    sliced_events = []
    def slicing_callback(events: dv.EventStore):
        extracted_events = extract_event_data(events)
        sliced_events.append(extracted_events)

    slicer.doEveryTimeInterval(timedelta(milliseconds=1), slicing_callback)
    while capture.isRunning():
        events = capture.getNextEventBatch()
        if events is not None:
            slicer.accept(events)
    
    # Concatenate all sliced event arrays into one array of shape (n, 4)
    if sliced_events:
        fused_array = np.concatenate(sliced_events, axis=0)
        return fused_array
    else:
        return np.array([])

    # return sliced_events

In [3]:
press_events = get_event_data('/Users/winirrr/Documents/EventBased_Project/raw_data/EB_data_1/press/dvSave-2024_06_27_16_22_11.aedat4')

In [4]:
df_press = pd.DataFrame(press_events, columns=["timestamp", "x", "y", "polarity"])

In [5]:
df_press.head()

,timestamp,x,y,polarity
0,1719472931617899,99,265,0
1,1719472931617900,95,452,1
2,1719472931617900,93,312,0
3,1719472931617901,90,410,0
4,1719472931617902,89,315,1


In [6]:
df_press['labels'] = 'press'

In [7]:
df_press.head()

,timestamp,x,y,polarity,labels
0,1719472931617899,99,265,0,press
1,1719472931617900,95,452,1,press
2,1719472931617900,93,312,0,press
3,1719472931617901,90,410,0,press
4,1719472931617902,89,315,1,press


In [8]:
no_press = get_event_data('/Users/winirrr/Documents/EventBased_Project/raw_data/EB_data_1/no_press/dvSave-2024_06_25_15_34_04.aedat4')

In [9]:
df_no_press = pd.DataFrame(no_press, columns=["timestamp", "x", "y", "polarity"])
df_no_press['labels'] = "no_press"
df_no_press.head()

,timestamp,x,y,polarity,labels
0,1719297244971248,40,476,0,no_press
1,1719297244971269,639,126,1,no_press
2,1719297244971285,585,132,1,no_press
3,1719297244971292,561,46,1,no_press
4,1719297244971293,560,51,0,no_press


In [10]:
df_press_500 = df_press[:500]

In [11]:
len(df_press_500)

500

In [12]:
df_no_press_500 = df_no_press[:500]

In [13]:
len(df_no_press_500)

500

In [17]:
import os
exp_0_df = pd.concat([df_press_500,df_no_press_500])


exp_data_dir = os.path.join("/Users/winirrr/Documents/EventBased_Project", 'exp_data')
os.makedirs(exp_data_dir, exist_ok=True)

exp_0_df.to_csv(os.path.join(exp_data_dir, 'exp_0_df.csv'))